**패키지 설치**
> datasets(huggingface 자연어처리 dataset)  
> konlpy(한글 자연어처리)  
> tqdm  
> PyKoSpacing(한글 띄어쓰기 보정 tool)  
> Py-Hanspell(네이버 한글 맞춤법 검사기 기반 한글 맞춤법 보정 tool)  
> sklearn


In [76]:
!pip install datasets
!pip install tqdm

#PyKoSpacing과 Py-hanspell은 깃허브 저장소에서 내려받아 커맨드에서 직접 설치 하였음
#https://github.com/ssut/py-hanspell
#https://github.com/haven-jeon/PyKoSpacing

# 데이터셋 준비

Naver Sentiment Movie Corpus(nsmc)
영화 리뷰를 별점을 기준으로 긍정(1), 부정(0)으로 구분한 데이터셋

In [1]:
from datasets import load_dataset

#nsmc 데이터셋 다운로드
dataset = load_dataset('nsmc')

Using custom data configuration default
Reusing dataset nsmc (C:\Users\ejcej\.cache\huggingface\datasets\nsmc\default\1.1.0\bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
#dataset 구조
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [3]:
# 첫 5개 문장 추출
for i in range(5):
    print(dataset['train'][i])
    
nsmc5_sents = [dataset['train'][idx]['document'] for idx in range(5)]
print(nsmc5_sents)

{'id': '9976970', 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}
{'id': '3819312', 'document': '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', 'label': 1}
{'id': '10265843', 'document': '너무재밓었다그래서보는것을추천한다', 'label': 0}
{'id': '9045019', 'document': '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', 'label': 0}
{'id': '6483659', 'document': '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', 'label': 1}
['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다']


Okt 형태소 분석기의 장점
- 속도가 빠르다
- 정제되지 않은 데이터에서 좋은 성능을 보인다

In [4]:
from konlpy.tag import Okt

okt = Okt()

#Okt 형태소 분석기로 문장을 형태소 단위의 token으로 분리
for sent in nsmc5_sents:
    word = [w for w,p in okt.pos(sent)]
    print(sent)
    print(word)
    print()

아 더빙.. 진짜 짜증나네요 목소리
['아', '더빙', '..', '진짜', '짜증나네요', '목소리']

흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
['흠', '...', '포스터', '보고', '초딩', '영화', '줄', '....', '오버', '연기', '조차', '가볍지', '않구나']

너무재밓었다그래서보는것을추천한다
['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '한', '다']

교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
['교도소', '이야기', '구먼', '..', '솔직히', '재미', '는', '없다', '..', '평점', '조정']

사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이뻐', '보였다']



위의 문장들을 보면 띄어쓰기와 오타로 인해 제대로 형태소 분석이 이루어지지 않고 있다.  
맞춤법 교정을 하고 다시 형태소 분석을 진행해보자.

In [5]:
from hanspell import spell_checker
from pykospacing import Spacing

spacing = Spacing()

tokenized_nsmc5_corpus = []
# 한글 전처리 패키지인 hanspell과 pykospacing 으로 전처리한 경우
for sent in nsmc5_sents:
    new_sent = spell_checker.check(sent).checked
    new_sent = spacing(new_sent)
    word = [w for w,p in okt.pos(sent)]
    
    tokenized_nsmc5_corpus.append(word)
    print(f'Before : {sent}')
    print(f'After : {new_sent}')
    print(word)
    print()

1/1 [==============================] - 1s 516ms/step
Before : 아 더빙.. 진짜 짜증나네요 목소리
After : 아 더빙.. 진짜 짜증 나네요 목소리
['아', '더빙', '..', '진짜', '짜증나네요', '목소리']

1/1 [==============================] - 0s 15ms/step
Before : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
After : 흠... 포스터 보고 초등학생 영화 줄.... 오버 연기조차 가볍지 않구나
['흠', '...', '포스터', '보고', '초딩', '영화', '줄', '....', '오버', '연기', '조차', '가볍지', '않구나']

1/1 [==============================] - 0s 15ms/step
Before : 너무재밓었다그래서보는것을추천한다
After : 너무 재밓었다 그래서 보는 것을 추천한다
['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '한', '다']

1/1 [==============================] - 0s 16ms/step
Before : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
After : 교도소 이야기구먼 .. 솔직히 재미는 없다.. 평점 조정
['교도소', '이야기', '구먼', '..', '솔직히', '재미', '는', '없다', '..', '평점', '조정']

1/1 [==============================] - 0s 15ms/step
Before : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
After : 사이먼 페그의 익살스러운 연기가 돋보였던 영화! 스파이더맨에서 늙어 보이기만 했던 커스틴 던스트가 너무나도 이 뻐 보였다
['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화'

# Bag Of Words(BoW)

In [7]:
# 단어 사전 생성
def build_vocab(tokenized_corpus):
    word2idx = dict()
    
    idx = 0
    for sent in tokenized_corpus:
        for word in sent:
            try: # 단어가 존재한다면
                tmp = word2idx[word]
            except: # 단어가 존재하지 않는다면
                word2idx[word] = idx
                idx += 1
    return word2idx

nsmc5_word2idx = build_vocab(tokenized_nsmc5_corpus)
print(len(nsmc5_word2idx))
print(nsmc5_word2idx)

55
{'아': 0, '더빙': 1, '..': 2, '진짜': 3, '짜증나네요': 4, '목소리': 5, '흠': 6, '...': 7, '포스터': 8, '보고': 9, '초딩': 10, '영화': 11, '줄': 12, '....': 13, '오버': 14, '연기': 15, '조차': 16, '가볍지': 17, '않구나': 18, '너': 19, '무재': 20, '밓었': 21, '다그': 22, '래서': 23, '보는것을': 24, '추천': 25, '한': 26, '다': 27, '교도소': 28, '이야기': 29, '구먼': 30, '솔직히': 31, '재미': 32, '는': 33, '없다': 34, '평점': 35, '조정': 36, '사이': 37, '몬페': 38, '그': 39, '의': 40, '익살스런': 41, '가': 42, '돋보였던': 43, '!': 44, '스파이더맨': 45, '에서': 46, '늙어': 47, '보이기만': 48, '했던': 49, '커스틴': 50, '던스트': 51, '너무나도': 52, '이뻐': 53, '보였다': 54}


In [9]:
# 단어 사전을 사용해 bow 변환
def bow(word_list, word2idx):
    bow_vector = [0] * len(word2idx)
    
    for word in word_list:
        try: # 단어 사전에 단어가 있다면
            bow_vector[word2idx[word]] += 1
        except: # 단어 사전에 존재하지 않는 단어(OVV)
            pass
    return bow_vector

print(tokenized_nsmc5_corpus[0])
print(bow(tokenized_nsmc5_corpus[0], nsmc5_word2idx))

print(tokenized_nsmc5_corpus[3])
print(bow(tokenized_nsmc5_corpus[3], nsmc5_word2idx))

['아', '더빙', '..', '진짜', '짜증나네요', '목소리']
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['교도소', '이야기', '구먼', '..', '솔직히', '재미', '는', '없다', '..', '평점', '조정']
[0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Sentiment Analysis(BoW)
긍/부정 분석

In [14]:
import numpy as np
from tqdm import tqdm

#1. 각 단어의 긍정 수치를 계산 ( 각 단어별 긍정 문장 수 / 단어가 등장한 문장 수)
#2. 한 문장의 긍정 수치는 그 문장을 구성하는 단어의 긍정 수치의 평균
#3. t 값은 전체 corpus의 긍정/부정 문장의 비율로 결정
#4. 긍정 수치가 t이상 -> 긍정, t미만 -> 부정

#nsmc 데이터 중 앞의 1000개
sents = [dataset['train'][idx]['document'] for idx in range(1000)]
labels = [dataset['train'][idx]['label'] for idx in range(1000)]

tokenized_nsmc1000_corpus = []

#1000개의 문장에 대해서 tokenized
for sent in sents: # tqdm 사용하지 않음. 이 코드의 경우 주피터에서는 너무 길어짐
    new_sent = spell_checker.check(sent).checked
    new_sent = spacing(new_sent)
    word = [w for w,p in okt.pos(new_sent)]
    
    tokenized_nsmc1000_corpus.append(word)

#vocab 생성
nsmc1000_word2idx = build_vocab(tokenized_nsmc1000_corpus)
nsmc1000_idx2word = {k:v for v,k in nsmc1000_word2idx.items()}
print(f'전체 단어 : {nsmc1000_word2idx}')
print()

1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 15ms/step
전체 단어 : {'아': 0, '더빙': 1, '..': 2, '진짜': 3, '짜증': 4, '나네요': 5, '목소리': 6, '흠': 7, '...': 8, '포스터': 9, '보고': 10, '초등학생': 11, '영화': 12, '줄': 13, '....': 14, '오버': 15, '연기': 16, '조차': 17, '가볍지': 18, '않구나': 19, '너무': 20, '재밓었다': 21, '그래서': 22, '보는': 23, '것': 24, '을': 25, '추천': 26, '한다': 27, '교도소': 28, '이야기': 29, '구먼': 30, '솔직히': 31, '재미': 32, '는': 33, '없다': 34, '평점': 35, '조정': 36, '사이먼': 37, '페그': 38, '의': 39, '익살스러운': 40, '가': 41, '돋보였던': 42, '!': 43, '스파이더맨': 44, '에서': 45, '늙어': 46, '보이기만': 47, '했던': 48, '커스틴': 49, '던스트': 50, '너무나도': 51, '이': 52, '뻐': 53, '보였다': 54, '막': 55, '걸음': 56, '마': 57, '뗀': 58, '3': 59, '세': 60, '부터': 61, '초등학교': 62, '1': 63, '학년': 64, '생인': 65, '8': 66, '살': 67, '용': 68, '.': 69, 'ㅋㅋㅋ': 70, '별': 71, '반': 72, '개도': 73, '아까': 74, '움': 75, '원작': 76, '긴장감': 77, '제대로': 78, '살려내': 79, '지': 80, '못': 81, '했다': 82, '아깝다': 83, '욕': 84, '나온': 85, '다': 86, '응경': 87, '길용우': 88, '연': 89, '기': 90, '생활': 91, '몇': 92, '년': 93, '

In [44]:
#vocab에 등장하는 단어별 긍정 수치 계산
pos_score = np.zeros(len(nsmc1000_word2idx)) # 긍정수치 numpy array
counts = np.zeros(len(nsmc1000_word2idx)) # 각 단어의 등장 횟수
num_pos = 0
num_token = 0

for idx, word_list in enumerate(tokenized_nsmc1000_corpus):
    _label = labels[idx] # 각 문장의 긍정(1), 부정(0) label
    
    for word in word_list:
        _idx = nsmc1000_word2idx[word]
        counts[_idx] += 1 # 각 단어의 등장 횟수
        pos_score[_idx] += _label # 각 단어가 긍정인 경우에 1
    
    num_pos += _label * len(word_list) # 긍정 문장인 경우에 추가
    num_token += len(word_list)
    
pos_score = pos_score/counts # 원소별로 나누기 연상

t = num_pos/num_token

print(f't값 : {num_pos/num_token}')
print()

t값 : 0.48651472604976165



In [45]:
#BoW 기반 5개 문장의 긍/부정 수치 계산 및 예측
for idx, word_list in enumerate(tokenized_nsmc1000_corpus[:5]):
    bow_vector = np.array(bow(word_list, nsmc1000_word2idx))
    weighted_bow_vector = bow_vector * pos_score
    sent_score = np.sum(weighted_bow_vector) / np.sum(bow_vector)
    
    result = '긍정' if sent_score >= t else '부정'
    answer = '긍정' if labels[idx] == 1 else '부정'

    print(f'{idx}번째 문장 : {sents[idx]}')
    print(f'정답 : {answer} | 예측 : {result} | 긍정수치 : {sent_score}')
    print()

0번째 문장 : 아 더빙.. 진짜 짜증나네요 목소리
정답 : 부정 | 예측 : 부정 | 긍정수치 : 0.185038732225433

1번째 문장 : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
정답 : 긍정 | 예측 : 긍정 | 긍정수치 : 0.6129973887174245

2번째 문장 : 너무재밓었다그래서보는것을추천한다
정답 : 부정 | 예측 : 부정 | 긍정수치 : 0.40889652815277666

3번째 문장 : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
정답 : 부정 | 예측 : 부정 | 긍정수치 : 0.27780186230688386

4번째 문장 : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
정답 : 긍정 | 예측 : 긍정 | 긍정수치 : 0.8045686931863756



In [47]:
#BoW 기반 전체 1000개 문장에 대한 문장의 긍/부정 예측 평가
pred = []
for idx, word_list in enumerate(tokenized_nsmc1000_corpus):
    bow_vector = np.array(bow(word_list, nsmc1000_word2idx))
    weighted_bow_vector = bow_vector * pos_score
    sent_score = np.sum(weighted_bow_vector) / np.sum(bow_vector)
    
    if sent_score >= t:
        _pred = 1
    else:
        _pred = 0
    
    pred.append(_pred)

pred = np.array(pred)
labels = np.array(labels)
print(f'정확도 : {np.sum(pred==labels)/len(pred)}')

정확도 : 0.986


## Sentence Similarity(BoW)
코사인 유사도를 사용한 단어유사도 

In [51]:
bow_vector_0 = np.array(bow(tokenized_nsmc1000_corpus[0], nsmc1000_word2idx))

cos_sim = []

for idx, word_list in enumerate(tokenized_nsmc1000_corpus[1:]):
    bow_vector_1 = np.array(bow(word_list, nsmc1000_word2idx))
    
    _cos_sim = np.dot(bow_vector_0, bow_vector_1)/(np.linalg.norm(bow_vector_0)*np.linalg.norm(bow_vector_1))
    cos_sim.append(_cos_sim)
    
cos_sim = np.array(cos_sim)

#[100, 300, 400, 200].argsort() -> [0,3,1,2] (크기 순으로 정렬했을 때 각 원소의 index가 반환됨)
top_5_idx = cos_sim.argsort()[-5:][::-1] #오름차순 정렬이라 뒤에서 5개를 가져옴

print(f'Query : {sents[0]}')
print('----top5----')
for idx in top_5_idx:
    print(sents[idx+1])

Query : 아 더빙.. 진짜 짜증나네요 목소리
----top5----
진짜 짜증나는 영화..
아
아 OOO기.. 이걸 본 내눈이 아깝다 ㅡㅡ 진짜 아놔 진짜 OOO기명작이네요
지루하다.. 지루해..
진짜 쓰레기영화.. 돈아까워 죽는줄 알았다.. 여러분 보지 마세요


# TF-IDF(Term Frequency - Inverse Document Frequency)

문서 내 각 단어의 빈도를 고려, 단어마다 가중치를 두는 방법  
흔히 등장하는 단어 = 낮은 가중치

In [53]:
def calculate_idf(word2idx, tokenized_corpus):
    df = np.zeros(len(word2idx))
    
    for sent in tokenized_corpus:
        sent_df = np.zeros(len(word2idx))
        
        for word in sent:  # 단어가 여러번 등장해도 1
            sent_df[word2idx[word]] = 1
        
        df += sent_df  # 각 단어들이 전체 문서에서 몇 개의 문장에 등장했는지 계살할 것
        
    idf = np.log(len(tokenized_corpus) / (1+df))  # 극단적으로 취해지는 가중치 방지하기 위해 log 씌움
    
    return idf

idf_nsmc5_corpus = calculate_idf(nsmc5_word2idx, tokenized_nsmc5_corpus)
print(idf_nsmc5_corpus)

# 여러번 등장하는 단어의 경우 가중치가 낮음

[0.91629073 0.91629073 0.51082562 0.91629073 0.91629073 0.91629073
 0.91629073 0.91629073 0.91629073 0.91629073 0.91629073 0.51082562
 0.91629073 0.91629073 0.91629073 0.51082562 0.91629073 0.91629073
 0.91629073 0.91629073 0.91629073 0.91629073 0.91629073 0.91629073
 0.91629073 0.91629073 0.91629073 0.91629073 0.91629073 0.91629073
 0.91629073 0.91629073 0.91629073 0.91629073 0.91629073 0.91629073
 0.91629073 0.91629073 0.91629073 0.91629073 0.91629073 0.91629073
 0.91629073 0.91629073 0.91629073 0.91629073 0.91629073 0.91629073
 0.91629073 0.91629073 0.91629073 0.91629073 0.91629073 0.91629073
 0.91629073]


In [59]:
def tf_idf(word_list, word2idx, idf):
    tf_vector = [0] * len(word2idx)
    
    for word in word_list:
        try:
            tf_vector[word2idx[word]] += 1
        except:
            pass
        
    tf_idf_vector = tf_vector * idf

    return tf_idf_vector

print(tokenized_nsmc5_corpus[0])
print(tf_idf(tokenized_nsmc5_corpus[0], nsmc5_word2idx, idf_nsmc5_corpus))

['아', '더빙', '..', '진짜', '짜증나네요', '목소리']
[0.91629073 0.91629073 0.51082562 0.91629073 0.91629073 0.91629073
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]


## Sentiment Analysis(TF-IDF)

In [61]:
idf_nsmc1000_corpus = calculate_idf(nsmc1000_word2idx, tokenized_nsmc1000_corpus)

#TF-IDF 기반 5개 문장의 긍/부정 수치 계산 및 예측
for idx, word_list in enumerate(tokenized_nsmc1000_corpus[:5]):
    tf_idf_vector = np.array(tf_idf(word_list, nsmc1000_word2idx, idf_nsmc1000_corpus))
    #tf_idf_vector = tf_idf_vector / np.linalg.norm(tf_idf_vector)
    weighted_tf_idf_vector = tf_idf_vector * pos_score
    #sent_score = np.mean(weighted_tf_idf_vector[tf_idf_vector != 0]) / scale_factor
    sent_score = np.sum(weighted_tf_idf_vector) / np.sum(tf_idf_vector)
    
    result = '긍정' if sent_score >= t else '부정'
    answer = '긍정' if labels[idx] == 1 else '부정'
    
    print(f'{idx}번째 문장 : {sents[idx]}')
    print(f'정답 : {answer} | 문장 분류 : {result} | 긍정수치 : {sent_score}')
    print()

0번째 문장 : 아 더빙.. 진짜 짜증나네요 목소리
정답 : 부정 | 문장 분류 : 부정 | 긍정수치 : 0.12615022286094732

1번째 문장 : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
정답 : 긍정 | 문장 분류 : 긍정 | 긍정수치 : 0.6606000688517063

2번째 문장 : 너무재밓었다그래서보는것을추천한다
정답 : 부정 | 문장 분류 : 부정 | 긍정수치 : 0.3583813514400063

3번째 문장 : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
정답 : 부정 | 문장 분류 : 부정 | 긍정수치 : 0.21428886262571972

4번째 문장 : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
정답 : 긍정 | 문장 분류 : 긍정 | 긍정수치 : 0.8950741769668523



In [63]:
# TF - IDF 기반 전체 1000개 문장에 대한 문장의 긍/부정 예측 평가
pred = []
for idx, word_list in enumerate(tokenized_nsmc1000_corpus):
    tf_idf_vector = np.array(tf_idf(word_list, nsmc1000_word2idx, idf_nsmc1000_corpus))
    weighted_tf_idf_vector = tf_idf_vector * pos_score
    sent_score = np.sum(weighted_tf_idf_vector) / np.sum(tf_idf_vector)
    
    if sent_score >= t:
        _pred = 1
    else:
        _pred = 0
    
    pred.append(_pred)
    
pred = np.array(pred)
labels = np.array(labels)

print(f'정확도 : {np.sum(pred == labels) / len(pred)}')

정확도 : 0.988


## Sentence Similarity(TF-IDF)

In [67]:
tf_idf_vector_0 = np.array(tf_idf(tokenized_nsmc1000_corpus[0], nsmc1000_word2idx, idf_nsmc1000_corpus))

cos_sim = []

for idx, word_list in enumerate(tokenized_nsmc1000_corpus[1:]):
    tf_idf_vector_1 = np.array(tf_idf(word_list, nsmc1000_word2idx, idf_nsmc1000_corpus))
    
    _cos_sim = np.dot(tf_idf_vector_0,tf_idf_vector_1)/(np.linalg.norm(tf_idf_vector_0)*np.linalg.norm(tf_idf_vector_1))
    cos_sim.append(_cos_sim)
    
cos_sim = np.array(cos_sim)

top_5_idx = cos_sim.argsort()[-5:][::-1] #오름차순 정렬이라 뒤에서 5개를 가져옴

print(f'Query : {sents[0]}')
print('----top5----')
for idx in top_5_idx:
    print(sents[idx+1])

Query : 아 더빙.. 진짜 짜증나네요 목소리
----top5----
진짜 짜증나는 영화..
아
더빙이 똥이야 ....
진짜..뭐냐 민국이나 지아는 더빙 더이상시키면안되겠네 더빙이 얼마나중요한데 이런애들을시켜데뷔도안한애들을 연예인이해도 뭐라고하는판에
아 정말짜증나네 절때보지말껄욕나옴


# sklearn 으로 BoW, TF-IDF벡터 생성하기

In [72]:
corpus = [dataset['train'][idx]['document'] for idx in range(1000)]
preprocessed_corpus = []

for sent in corpus:
    new_sent = spell_checker.check(sent).checked
    new_sent = spacing(new_sent)
    preprocessed_corpus.append(new_sent)

1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 16ms/step


In [74]:
from sklearn.feature_extraction.text import CountVectorizer

vector_bow = CountVectorizer()
vector_bow.fit(preprocessed_corpus)

print(f'BoW vocab : {vector_bow.vocabulary_}')
print(f'BoW vocab : {vector_bow.transform([preprocessed_corpus[0]])}')
print(f'BoW vocab : {vector_bow.transform([preprocessed_corpus[0]]).toarray()}')

BoW vocab : {'더빙': 1223, '진짜': 4373, '짜증': 4392, '나네요': 826, '목소리': 1768, '포스터': 4749, '보고': 2058, '초등학생': 4481, '영화': 3301, '오버': 3388, '연기조차': 3251, '가볍지': 230, '않구나': 2974, '너무': 971, '재밓었다': 3974, '그래서': 636, '보는': 2066, '것을': 440, '추천한다': 4525, '교도소': 567, '이야기구먼': 3649, '솔직히': 2602, '재미는': 3917, '없다': 3144, '평점': 4737, '조정': 4155, '사이먼': 2354, '페그의': 4715, '익살스러운': 3684, '연기가': 3235, '돋보였던': 1259, '스파이더맨에서': 2672, '늙어': 1089, '보이기만': 2100, '했던': 4928, '커스틴': 4575, '던스트가': 1228, '너무나도': 973, '보였다': 2096, '걸음마': 424, '3세부터': 80, '초등학교': 4480, '1학년생인': 50, '8살': 105, 'ㅋㅋㅋ': 190, '개도': 359, '아까움': 2850, '원작의': 3524, '긴장감을': 735, '제대로': 4118, '살려내': 2373, '못했다': 1796, '아깝다': 2856, '나온': 865, '응경': 3594, '길용우': 741, '연기생활이': 3247, '년인지': 989, '정말': 4083, '발로': 1966, '해도': 4903, '그것보단': 621, '낫겠다': 934, '납치': 933, '감금만': 276, '반복': 1951, '드라마는': 1338, '가족도': 248, '연기': 3234, '못하는': 1788, '사람만': 2323, '모였네': 1763, '액션이': 3044, '없는데도': 3142, '재미있는': 3935, '되는': 1303, '이렇게': 3625, '평점이': 4

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer

vector_tf_idf = TfidfVectorizer()
vector_tf_idf.fit(preprocessed_corpus)

print(f'TF-IDF vocab : {vector_tf_idf.vocabulary_}')
print(f'TF-IDF vocab : {vector_tf_idf.transform([preprocessed_corpus[0]])}')
print(f'TF-IDF vocab : {vector_tf_idf.transform([preprocessed_corpus[0]]).toarray()}')

TF-IDF vocab : {'더빙': 1223, '진짜': 4373, '짜증': 4392, '나네요': 826, '목소리': 1768, '포스터': 4749, '보고': 2058, '초등학생': 4481, '영화': 3301, '오버': 3388, '연기조차': 3251, '가볍지': 230, '않구나': 2974, '너무': 971, '재밓었다': 3974, '그래서': 636, '보는': 2066, '것을': 440, '추천한다': 4525, '교도소': 567, '이야기구먼': 3649, '솔직히': 2602, '재미는': 3917, '없다': 3144, '평점': 4737, '조정': 4155, '사이먼': 2354, '페그의': 4715, '익살스러운': 3684, '연기가': 3235, '돋보였던': 1259, '스파이더맨에서': 2672, '늙어': 1089, '보이기만': 2100, '했던': 4928, '커스틴': 4575, '던스트가': 1228, '너무나도': 973, '보였다': 2096, '걸음마': 424, '3세부터': 80, '초등학교': 4480, '1학년생인': 50, '8살': 105, 'ㅋㅋㅋ': 190, '개도': 359, '아까움': 2850, '원작의': 3524, '긴장감을': 735, '제대로': 4118, '살려내': 2373, '못했다': 1796, '아깝다': 2856, '나온': 865, '응경': 3594, '길용우': 741, '연기생활이': 3247, '년인지': 989, '정말': 4083, '발로': 1966, '해도': 4903, '그것보단': 621, '낫겠다': 934, '납치': 933, '감금만': 276, '반복': 1951, '드라마는': 1338, '가족도': 248, '연기': 3234, '못하는': 1788, '사람만': 2323, '모였네': 1763, '액션이': 3044, '없는데도': 3142, '재미있는': 3935, '되는': 1303, '이렇게': 3625, '평점이'